# Effluent Totals

Notebook that can be turned into a python script to caluclate zonal stats for effluent totals for each watershed and then connect them to pour points (taken from Jared's python and then R script).

The watersheds are in different CRS and thus cannot be stacked. They will be converted to espg 54009, which will make some coastal issues, but on the whole this is the best we can do.

By Cascade Tuholske 2019-11-11

**This is Jared's code adpoted**

This works but be sure to change the file paths and names for FIO and N accordingly - CPT 2020.02.02

**UPDATED 2020-03-23** <br>
This needs to be run for total N and each treatment type (open, septic, and treated) for
watershed-level attribution <br>
Country-level data added

In [ ]:
#### Dependencies
from rasterstats import zonal_stats, gen_zonal_stats
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import os

In [ ]:
#### File Paths and Files
##############################################################################################################

DATA_IN = '/home/cascade/projects/wastewater/data/'
MASK_FN = DATA_IN+'interim/inlandwatersheds_mask.tif'

#### Make Masked Effluent Rasters 
##############################################################################################################

def mask_effluent(effluent_fn, mask_fn, out_fn):
    
    """Function opens effluent raster and masks inland watershed pixels\
    Args:
        effluent_fn = effluent raster fn
        mask_fn = mask raster fn
        out_fn = file name out
    """
    
    # Open effluent raster
    rst = rasterio.open(effluent_fn)
    meta = rst.meta
    
    # Update Data Type
    meta.update({'dtype' : 'float64'}) 
    band = rst.read(1)
    band = band.astype('float64')
    
    # mask inland watersheds
    mask = rasterio.open(mask_fn).read(1)
    mask[mask == 1] = 2 # revalue mask so inland watersheds are = 0 
    mask[mask == 0] = 1
    mask[mask == 2] = 0
    
    band_out = band * mask 

    # Save new data type and mask out
    with rasterio.open(out_fn, 'w', **meta) as dst:
        dst.write(band_out, 1)
    
    print('Done', out_fn)

# effluent rsts
effluent_rsts = [DATA_IN+'interim/effluent_N.tif', DATA_IN+'interim/effluent_N_treated.tif', 
                 DATA_IN+'interim/effluent_N_septic.tif', DATA_IN+'interim/effluent_N_open.tif']

# Make masked rasters
for rst in effluent_rsts:
    
    # Get data type
    data = rst.split('interim/')[1].split('.')[0]
    print(data)

    # Raster Mask 64 
    print('Starting mask64', rst)
    rst_out = DATA_IN+'interim/'+data+'_mask64.tif'
    mask_effluent(rst, MASK_FN, rst_out)

#### Zonal Stats
##############################################################################################################




























# Watersheds

In [ ]:
### File Paths on CPT Home
# Basins have already been reprojected to EPSG 59004
data = 'N_treated' # name of data: N, open, septic, treated 
data_type = 'N' # FIO or N in directory name
fn_in = "effluent_"+data
data_dir = "/home/cascade/projects/wastewater/data/interim/"
data_out =  "/home/cascade/projects/wastewater/data/interim/"+data_type+"_effluent_output/"
basins_dir = data_dir+"basins_crs/"
shps = [os.path.join(basins_dir, fn) for fn in os.listdir(basins_dir) if fn.endswith("59004.shp")]
effluent_fn = os.path.join(data_dir, fn_in+'.tif')

In [ ]:
# First need to recast rasters as float64

rst = rasterio.open(effluent_fn)
meta = rst.meta
meta.update({'dtype' : 'float64'}) 
band = rst.read(1)
band = band.astype('float64')

with rasterio.open(data_dir+fn_in+'64.tif', 'w', **meta) as dst:
    dst.write(band, 1)


In [ ]:
#Run Zonal Stats
effluent_fn = os.path.join(data_dir, fn_in+'64.tif') # update for float64

feature_list = []

for shp_fn in shps:
    watersheds = gpd.read_file(shp_fn)
    zs_feats = zonal_stats(watersheds, effluent_fn, stats="sum count", geojson_out=True)
    feature_list.extend(zs_feats)
    print(shp_fn, ' is done')
    
zgdf = gpd.GeoDataFrame.from_features(feature_list, crs=watersheds.crs)
zgdf = zgdf.rename(columns={'sum': 'effluent'})
zgdf.effluent = zgdf.effluent.fillna(0)

In [ ]:
zgdf.to_file(data_out+'effluent_'+data+'_watersheds.shp')

In [ ]:
zgdf.to_csv(data_out+'effluent_'+data+'_watersheds.csv')

#### Check them

In [ ]:
check = gpd.read_file(data_out+'effluent_'+data+'_watersheds.shp')

In [ ]:
check.sort_values(['effluent'], ascending = False).head(5)

# Country-level
Completed by Cascade Tuholske 2020.03.23

In [ ]:
# Basins have already been reprojected to EPSG 59004
data = 'N_septic' # name of data: N, open, septic, treated 
data_type = 'N' # FIO or N in directory name
fn_in = "effluent_"+data
data_dir = "/home/cascade/projects/wastewater/data/interim/"
data_out =  "/home/cascade/projects/wastewater/data/interim/"+data_type+"_effluent_output/"
countries_shps_fn = 'world_vector.shp'
effluent_fn = os.path.join(data_dir, fn_in+'.tif')

In [ ]:
# First need to recast rasters as float64
rst = rasterio.open(effluent_fn)
meta = rst.meta
meta.update({'dtype' : 'float64'}) 
band = rst.read(1)
band = band.astype('float64')

with rasterio.open(data_dir+fn_in+'64.tif', 'w', **meta) as dst:
    dst.write(band, 1)

In [ ]:
# Load files

effluent_fn = os.path.join(data_dir, fn_in+'64.tif') # update for float64
countries = gpd.read_file(data_dir+countries_shps_fn) # load countries

In [ ]:
#Run Zonal Stats
zs_feats = zonal_stats(countries, effluent_fn, stats="sum count", geojson_out=True)

In [ ]:
# write to a dataframe  
zgdf = gpd.GeoDataFrame.from_features(zs_feats, crs=countries.crs)
zgdf = zgdf.rename(columns={'sum': 'effluent'})
zgdf.effluent = zgdf.effluent.fillna(0)

In [ ]:
zgdf.head()

In [ ]:
# write out

zgdf.to_file(data_out+'effluent_'+data+'_countries.shp')

#### Check them

In [ ]:
check = gpd.read_file(data_out+'effluent_'+data+'_countries.shp')

In [ ]:
check.sort_values(['effluent'], ascending = False).head(5)

# Old CODE

#### Dependencies

In [ ]:
from rasterstats import zonal_stats, gen_zonal_stats
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import os
import matplotlib.pyplot as plt

#### Load Files

In [ ]:

### File Paths on CPT Home
# Basins have already been reprojected to 59004
data_dir = "/Users/cascade/Github/wastewater_ohi/data/interim/"
data_out =  "/Users/cascade/Github/wastewater_ohi/data/processed/FIO_effluent_output/"
basins_dir = os.path.join(data_dir, "basins_crs/")
shps = [os.path.join(basins_dir, fn) for fn in os.listdir(basins_dir) if fn.endswith("59004.shp")]

In [ ]:
### This is from my ERL paper, it should work for the GHS
# https://github.com/cascadet/AfricaUrbanPop/blob/master/notebooks/jupyter/ERL19/Step4_Zonal_Stats.ipynb 
# Update - this dict is the correct for espg 54009, which is not in fiona, but works fine.
# see this for more details: https://epsg.io/54009

new_crs = {'proj': 'moll', 'lon_0': 0, 'x_0': 0, 'y_0': 0, 'ellps': 'WGS84', 'units': 'm', 'no_defs': True}


#### Get Geom to write out files

In [ ]:
# Get geometry for watersheds to write out 

geom_out = []
basin_id_list = []
area_list = []
for shp_fn in shps: 
    shp_fn = gpd.read_file(shp_fn)# .to_crs(new_crs) # switches them all to espg 54009 ... cpt 2020.01.24 not needed
    basin_id = shp_fn['basin_id']
    geom = shp_fn['geometry']
    area = shp_fn['area']
    basin_id_list.extend(basin_id)
    geom_out.extend(geom)
    area_list.extend(area)

In [ ]:
# Make Dataframe to write out

out_shape = gpd.GeoDataFrame()
out_shape['geometry'] = geom_out
out_shape['basin_id'] = basin_id_list
out_shape['area'] = area_list

In [ ]:
out_shape.head()

## Run zonal stats

Jared's code does not seem to be working. Producing -inf and NAs, going to try my code from the Africa project

**Be sure to switch l, m, h files since I did not write it as a loop**

In [ ]:
### 2020.01.24 CPT --- make 

col = 'FIO'
effluent_fn = os.path.join(data_dir, "effluent_FIO.tif")
#output_fn = os.path.join(data_out, "FIO_sep0_effluent_watersheds_all.shp")

#### Check CRS first

In [ ]:
### Check crs of .tif and shape files

shp_test = gpd.read_file(shps[0])
print('Shape crs \n ', shp_test.crs)
rst_test = rasterio.open(effluent_fn)
print('Raster crs \n', rst_test.crs)

In [ ]:
shp_test.head()

In [ ]:
def zone_stat(raster, band, polygon, stats, touched):
    """
    This function will calculate the zonal stats for each polygon within a raster
    requires gpd_df, raster, object and nodata value
    
    Args: raster = input raster
          band = band of raster
          polygon = polygons to calc zonal stats 
          stats = stat to calculate as string
          touched = True or False, to include pixels intersected w/ polygons
    """
    
    band = raster.read(band)
    band[band < 0] = 0 # Fix missing data
    zone_stat = zonal_stats(polygon, band, affine=raster.meta['transform'], 
                            nodata = -3.4e+38, stats = stats, all_touched = touched)
    return zone_stat

### run loop

In [ ]:
### Calc Zonal Stats
### Running sontal stats with all touched = True https://pythonhosted.org/rasterstats/manual.html#statistics

rst = rasterio.open(effluent_fn) # Open raster
feature_list = []

for shp_fn in shps:
    watersheds = gpd.read_file(shp_fn).to_crs(new_crs) 
    zs_feats = zone_stat(rst, 1, watersheds, 'sum', True)
    feature_list.extend(zs_feats)
    print('One shape is done')
print('finished!')

In [ ]:
# Remove Nans and set to log scale if desired 

out_shape[col] = pd.DataFrame.from_dict(feature_list)
out_shape[col] = out_shape[[col]].replace(0, np.nan) # Set zeros to NAN, can run as log if needed
out_shape[col] = out_shape[col].fillna(0)

In [ ]:
out_shape.head()

#### Save

In [ ]:
# save out a sub-set

# Dropping all the tiny tiny fractional watersheds 

out_shape_sub = out_shape[out_shape[col] >0]
print(len(out_shape_sub))

In [ ]:
out_shape_sub.head()

In [ ]:
# Write out 
out_shape.to_file(data_out+col+'_effluent_watersheds.shp')
out_shape_sub.to_file(data_out+col+'_effluent_watersheds_sub.shp')

# Pour Points

In [ ]:
pour_points = gpd.read_file(data_dir+'global_plume_2007_2010.shp') # Open Pour Points

In [ ]:
# switch crs to match ocean mask
print(pour_points.crs)
pour_points = pour_points.to_crs({'init': 'epsg:4326'})
print(pour_points.crs)

In [ ]:
pour_points.head()

In [ ]:
## Join Watershed Effluent Values to Pour Points
print(len(pour_points))
pp_merge_all = out_shape.drop(columns = 'geometry')
pp_merge_all = pd.merge(pp_merge_all, pour_points, on = 'basin_id', how = 'inner') # <<--- one gets dropped
print(len(pp_merge_all))

In [ ]:
pp_merge_all.head()

In [ ]:
## Join Watershed Effluent Values to Pour Points
print(len(pour_points))
pp_merge_sub = out_shape_sub.drop(columns = 'geometry')
pp_merge_sub = pd.merge(pp_merge_sub, pour_points, on = 'basin_id', how = 'inner') # <<--- one gets dropped
print(len(pp_merge_sub))

In [ ]:
pp_merge_sub.head()

In [ ]:
### Write out pour points

pp_merge_all.to_file(data_out+col+'_pour_point_totals_all.shp')
pp_merge_sub.to_file(data_out+col+'_pour_point_totals_sub.shp')

# Pour point data for plumes

From the pilot, we need pour points with the following columns plus geometry in the following order:
gemometry, basin_id, FIO (e.g. effluent) and area.

I am going to use the sub-pours file or effluent with >0 to reduce plume processing time
CPT 2020.01.28

In [ ]:
final_pours = gpd.GeoDataFrame()
final_pours['geometry'] = pp_merge_sub['geometry']
final_pours['basin_id'] = pp_merge_sub['basin_id']
final_pours['effluent'] = pp_merge_sub['FIO'] ## Has to be named effluent based on plume code
final_pours['area'] = pp_merge_sub['area']
final_pours['count'] = np.nan
final_pours.head()


In [ ]:
final_pours.to_file(data_out+col+'_pourpoints_final.shp')

In [ ]:
FIO_500 = final_pours[:500]

In [ ]:
FIO_500.to_file(data_out+col+'_pourpoints_500.shp')

In [ ]:
FIO_500.head()

# Test of FIO

In [ ]:
test5000 = gpd.read_file(data_out+'FIO_pourpoints_final.shp')

In [ ]:
test5000 = test5000[:5000].copy()

In [ ]:
test5000['area'] = np.nan

In [ ]:
test5000.head()

In [ ]:
test5000.crs = {'init' :'epsg:4326'}

In [ ]:
test5000.to_file('~/Desktop/test5000.shp')

# Old CODE

## Write out top 100 Watersheds

#### Totals

In [ ]:
pp_merge_sub_sort = pp_merge_sub.sort_values(by = 'Nitrogen', ascending = False)[0:100]

In [ ]:
pp_merge_sub_sort.to_file(data_out+'Nitrogen_pour_point_totals_sub100.shp')

In [ ]:
out_shape_sub_sort = out_shape_sub.sort_values(by = 'Nitrogen', ascending = False)[0:100]
out_shape_sub_sort.to_file(data_out+'Nitrogen_effluent_watersheds_sub100.shp')

#### By area

In [ ]:
pp_merge_sub_sort = pp_merge_sub.sort_values(by = 'Nitrogen_area', ascending = False)[0:100]

In [ ]:
pp_merge_sub_sort.to_file(data_out+'Nitrogen_pour_point_area_sub100.shp')

In [ ]:
out_shape_sub_sort = out_shape_sub.sort_values(by = 'Nitrogen_area', ascending = False)[0:100]
out_shape_sub_sort.to_file(data_out+'Nitrogen_effluent_watersheds_area_sub100.shp')

#### Area and pct

In [ ]:
# Effluent by pct

out_shape['Nitrogen_pct'] = out_shape['Nitrogen'] / out_shape['Nitrogen'].sum() 

In [ ]:
# Effluent by pct


out_shape['Nitrogen_area'] = out_shape['Nitrogen'] / out_shape['area']

print('done')

In [ ]:
out_shape.head()